# Snorkel on IMDb

In [17]:
%pip install snorkel datasets scikit-learn pandas

## 1. Load and Explore the IMDb Dataset

In [18]:
from datasets import load_dataset
import pandas as pd

# Load 2000 training and 500 test examples for speed​

imdb = load_dataset("imdb")
train = pd.DataFrame(imdb["train"].select(range(2000)))
test = pd.DataFrame(imdb["test"].select(range(500)))
print("Train size:", len(train), "Test size:", len(test))
train.head()

Train size: 2000 Test size: 500


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


## 2. Preprocess Text

In [19]:
import re
def clean_text(text):
    text = re.sub(r"<br\s*/?>", " ", text)
    text = re.sub(r"[^\w\s']", "", text)
    return text.lower()

train["text"] = train["text"].apply(clean_text)
test["text"] = test["text"].apply(clean_text)

## 3. Define Labeling Functions (LFs)

Create simple heuristics:

+ **LF_positive**: labels text as positive if it contains strong positive words.​​
+ **LF_negative**: labels text as negative if it contains strong negative words
+ **LF_exclaim**: positive if contains “!” more than 2 times

In [20]:
from snorkel.labeling import labeling_function, LFAnalysis
# from snorkel.labeling import LabelModel
from snorkel.labeling.model.label_model import LabelModel

ABSTAIN, NEG, POS = -1, 0, 1
positive_words = {"great","excellent","amazing","wonderful","best","fantastic"}
negative_words = {"bad","terrible","awful","worst","boring","poor"}

@labeling_function()
def lf_positive(x):
    return POS if any(w in x.text.split() for w in positive_words) else ABSTAIN

@labeling_function()
def lf_negative(x):
    return NEG if any(w in x.text.split() for w in negative_words) else ABSTAIN

@labeling_function()
def lf_exclaim(x):
    return POS if x.text.count("!") > 2 else ABSTAIN
lfs = [lf_positive, lf_negative, lf_exclaim]

#### Analyze LF Coverage & Conflicts

In [21]:
from snorkel.labeling import PandasLFApplier
applier = PandasLFApplier(lfs)
L_train = applier.apply(train)
LFAnalysis(L_train, lfs).lf_summary()

100%|██████████| 2000/2000 [00:00<00:00, 4400.20it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
lf_positive,0,[1],0.3210,0.1795,0.1795
lf_negative,1,[0],0.5635,0.1795,0.1795
lf_exclaim,2,[],0.0000,0.0000,0.0000


## 4. Train the LabelModel

In [22]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

# Get probabilistic labels​
train_probs = label_model.predict_proba(L_train)
train_preds = label_model.predict(L_train)

100%|██████████| 500/500 [00:00<00:00, 631.52epoch/s]


## 5. Train an End-to-End Classifier

Use a simple logistic regression on TF-IDF features:

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Vectorize​

vectorizer = TfidfVectorizer(max_features=5_000)
X_train = vectorizer.fit_transform(train["text"])
y_train = train_preds

# Fit classifier​

clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)

# Evaluate on test set​

X_test = vectorizer.transform(test["text"])
y_test = test["label"]
preds = clf.predict(X_test)
print(classification_report(y_test, preds, target_names=["neg","pos","neu"]))

              precision    recall  f1-score   support

         neg       0.00      0.00      0.00         0
         pos       1.00      0.81      0.90       500
         neu       0.00      0.00      0.00         0

    accuracy                           0.81       500
   macro avg       0.33      0.27      0.30       500
weighted avg       1.00      0.81      0.90       500



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 6. Evaluate Weak Supervision vs. Fully Supervised

For comparision, train the same classifier on 2,000 true labels

In [24]:
clf_fs = LogisticRegression(max_iter=200)
clf_fs.fit(X_train, train["label"])
fs_preds = clf_fs.predict(X_test)
print("Fully supervised performance:")
print(classification_report(y_test, fs_preds, target_names=["neg","pos"]))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(0)

+ Visualize confusion matrices and coverage-accuracy trade-off curves